# Callback Analysis

In [1]:
import sys
sys.path.append("..")

import os
import seaborn as sns
from google.cloud import bigquery
from src import Flow
from matplotlib import pyplot as plt




credential_path = "/home/kerri/bigquery-jaya-consultant-cosmic-octane-88917-c46ba9b53a3b.json"
assert os.path.exists(credential_path)
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path
project_id = 'cosmic-octane-88917'
client = bigquery.Client(project=project_id)


In [2]:
sql = """ WITH subset AS (
SELECT *
FROM `cosmic-octane-88917.analytics_us._VW_CallEvent`
WHERE FlowName = '{0}'
AND EXTRACT(DATE FROM TimeStamp) BETWEEN '2020-06-01' AND '2020-12-01'
AND CallingNumber != 'Restricted'
),

callbacks AS (
SELECT DISTINCT CallingNumber,
       CASE WHEN LEFT(CallingNumber, 4) in ('+1800%') THEN True ELSE False END AS TollFreeNumber,
       EXTRACT(DATE FROM c1.TimeStamp) date,
       c1.TimeStamp, 
       c1.SessionId first_session_id, 
       c1.CallAnswerIndicator,
       c1.CallDuration first_call_duration,
       c2.TimeStamp, 
       c2.SessionId second_session_id,
       c2.CallAnswerIndicator,
       c2.CallDuration second_call_duration,
       ABS(TIMESTAMP_DIFF(c1.TimeStamp, c2.TimeStamp, DAY)) day_diff
FROM subset c1
INNER JOIN subset c2
USING(CallingNumber)
WHERE c1.SessionId != c2.SessionId
AND c1.TimeStamp < c2.TimeStamp
)


SELECT date,  
       TollFreeNumber, 
       AVG(day_diff) mean_days_between_calls, 
       STDDEV(day_diff) std_days_between_calls, 
       COUNT(*) count,
       AVG(first_call_duration) mean_first_call_duration,
       STDDEV(first_call_duration) std_first_call_duration,
       AVG(second_call_duration) mean_second_call_duration,
       STDDEV(second_call_duration) std_second_call_duration
FROM callbacks
GROUP BY date, TollFreeNumber
ORDER BY date, TollFreeNumber
"""


In [3]:
def plot_callbacks(df):
    fig, axs = plt.subplots(ncols=2, nrows=2, figsize=(15,15))
    sns.lineplot(data=df, x='date', y='count', hue='TollFreeNumber', ax=axs[0][0]).set(title='Count vs Date')
    sns.lineplot(data=df, x='date', y='mean_days_between_calls', hue='TollFreeNumber', ax=axs[0][1]).set(title='Average Days Between Calls vs Date')
    sns.lineplot(data=df, x='date', y='mean_first_call_duration', hue='TollFreeNumber', ax=axs[1][0]).set(title='Average Duration of First Call vs Date')
    sns.lineplot(data=df, x='date', y='mean_second_call_duration', hue='TollFreeNumber', ax=axs[1][1]).set(title='Average Duration of Second Call vs Date')

In [7]:
df = client.query(sql.format("United Kingdom - Customer Service")).to_dataframe()
plot_callbacks(df)

PermissionDenied: 403 request failed: the user does not have 'bigquery.readsessions.create' permission for 'projects/cosmic-octane-88917'

## Callbacks - United Kingdom - Customer Service

The above plot show characteristics of phone numbers that have appeared in more than one SessionId. The top left is the count of how many of these repeat numbers called for the first time on that date. The top right plot shows the number of days between the two sessionIds and the bottom two plots display the duration in minutes for both the first call and the second. 

**TollFreeNumbers** is a subgroup of phone numbers that start with '+1800'

### Takeaways

- There is a ver pronounced downward trend in the number of days between calls, although this might not be real. It could be an affect of callbacks that haven't occured yet.
- There is a downward trend in the count of callbacks, although this might not be real. It could be an affect of callbacks that haven't occured yet. With that gap in information in mind we could say that the number of callbacks occuring is probably NOT trending upwards.
- The second call started out with a similar avgerage duration in June-July but in the most recent months it's moved up faster than the first call
- The toll free numbers have significantly shorter call durations
- the toll free numbers followed the downward trend of but consistantly have more time between calls 

In [ ]:
df = client.query(sql.format("Germany - Customer Service")).to_dataframe()
plot_callbacks(df)

## Callbacks - Germany - Customer Service

The above plot show characteristics of phone numbers that have appeared in more than one SessionId. The top left is the count of how many of these repeat numbers called for the first time on that date. The top right plot shows the number of days between the two sessionIds and the bottom two plots display the duration in minutes for both the first call and the second. 

**TollFreeNumbers** is a subgroup of phone numbers that start with '+1800'

### Takeaways

- There is a slight downward trend in the number of days between calls
- First and Second call duration appear to be independent. Again the call duration appears to be affected by the date.
- There we no toll free numbers

In [ ]:
df = client.query(sql.format("Spain - Customer Service")).to_dataframe()
plot_callbacks(df)

## Callbacks - Spain - Customer Service

The above plot show characteristics of phone numbers that have appeared in more than one SessionId. The top left is the count of how many of these repeat numbers called for the first time on that date. The top right plot shows the number of days between the two sessionIds and the bottom two plots display the duration in minutes for both the first call and the second. 

**TollFreeNumbers** is a subgroup of phone numbers that start with '+1800'

### Takeaways

- The presence of callbacks is sparce in this dataset
- The first and second call duration seems to be independent of each other but dependent on the date
- This dataset did not have all toll free numbers

In [ ]:
df = client.query(sql.format("France - Customer Service")).to_dataframe()
plot_callbacks(df)

## Callbacks - Spain - Customer Service

The above plot show characteristics of phone numbers that have appeared in more than one SessionId. The top left is the count of how many of these repeat numbers called for the first time on that date. The top right plot shows the number of days between the two sessionIds and the bottom two plots display the duration in minutes for both the first call and the second. 

**TollFreeNumbers** is a subgroup of phone numbers that start with '+1800'

### Takeaways

- The presence of callbacks is sparce in this dataset
- The first and second call duration seems to be independent of each other but dependent on the date
- This dataset had only a few toll free numbers that appeared in Sept. 

In [ ]:
df = client.query(sql.format("Italy - Customer Service")).to_dataframe()
plot_callbacks(df)

## Callbacks - Italy - Customer Service

The above plot show characteristics of phone numbers that have appeared in more than one SessionId. The top left is the count of how many of these repeat numbers called for the first time on that date. The top right plot shows the number of days between the two sessionIds and the bottom two plots display the duration in minutes for both the first call and the second. 

**TollFreeNumbers** is a subgroup of phone numbers that start with '+1800'

### Takeaways

- The presence of callbacks in this dataset is spase.
- There appears to be a downward trend in days between calls in the toll free number subgroup. However, given the sparse data it's hard to vaildate that this is true.
- The first and second call duration seems to be independent of each other but dependent on the date

# Solutran

In [ ]:
df = client.query(sql.format("HBP MainMenu")).to_dataframe()
plot_callbacks(df)

## Callbacks - HBP MainMenu

The above plot show characteristics of phone numbers that have appeared in more than one SessionId. The top left is the count of how many of these repeat numbers called for the first time on that date. The top right plot shows the number of days between the two sessionIds and the bottom two plots display the duration in minutes for both the first call and the second. 

**TollFreeNumbers** is a subgroup of phone numbers that start with '+1800'

### Takeaways

- There were not callbacks for this flow

In [ ]:
df = client.query(sql.format("Post Call Survey")).to_dataframe()
plot_callbacks(df)

## Callbacks - Post Call Survey

The above plot show characteristics of phone numbers that have appeared in more than one SessionId. The top left is the count of how many of these repeat numbers called for the first time on that date. The top right plot shows the number of days between the two sessionIds and the bottom two plots display the duration in minutes for both the first call and the second. 

**TollFreeNumbers** is a subgroup of phone numbers that start with '+1800'

### Takeaways

- Examples of toll free numbers are very sparse
- Both the first and second call durations are very consistant. They show little variation between them.

In [ ]:
df = client.query(sql.format("HBP Order")).to_dataframe()
plot_callbacks(df)

## Callbacks - HBP Order

The above plot show characteristics of phone numbers that have appeared in more than one SessionId. The top left is the count of how many of these repeat numbers called for the first time on that date. The top right plot shows the number of days between the two sessionIds and the bottom two plots display the duration in minutes for both the first call and the second. 

**TollFreeNumbers** is a subgroup of phone numbers that start with '+1800'

### Takeaways

- Examples of toll free numbers are very sparse
- The majority of the first call durations are zero indicating that were most likely a transfer that moved the user out of the flow quickly. However, there appears to have been a change in October that has caused the calls to have durations.
- The second calls have had durations since June. However, we see large increase in average call duration around the same time as behavioral change mentioned above. 

In [ ]:
df = client.query(sql.format("HBP Transfer TTEC OffShore")).to_dataframe()
plot_callbacks(df)

## Callbacks - HBP Transfer TTEC OffShore

The above plot show characteristics of phone numbers that have appeared in more than one SessionId. The top left is the count of how many of these repeat numbers called for the first time on that date. The top right plot shows the number of days between the two sessionIds and the bottom two plots display the duration in minutes for both the first call and the second. 

**TollFreeNumbers** is a subgroup of phone numbers that start with '+1800'

### Takeaways

- The first call and second call durations are very similar for the non-toll-free numbers
- The second call durations are consistantly longer for the toll free numbers

In [ ]:
df = client.query(sql.format("HBP Transfer Concentrix OffShore")).to_dataframe()
plot_callbacks(df)

## Callbacks - HBP Transfer Concentrix OffShore

The above plot show characteristics of phone numbers that have appeared in more than one SessionId. The top left is the count of how many of these repeat numbers called for the first time on that date. The top right plot shows the number of days between the two sessionIds and the bottom two plots display the duration in minutes for both the first call and the second. 

**TollFreeNumbers** is a subgroup of phone numbers that start with '+1800'

### Takeaways

- The first call and second call durations are very similar for the non-toll-free numbers
- Toll free numbers data is very sparse

# American Water

In [ ]:
df = client.query(sql.format("National Flow")).to_dataframe()
plot_callbacks(df)

## Callbacks - National Flow

The above plot show characteristics of phone numbers that have appeared in more than one SessionId. The top left is the count of how many of these repeat numbers called for the first time on that date. The top right plot shows the number of days between the two sessionIds and the bottom two plots display the duration in minutes for both the first call and the second. 

**TollFreeNumbers** is a subgroup of phone numbers that start with '+1800'

### Takeaways

- Second call - non-toll-free durations are extremely consistant
- In general the toll free numbers appear to have a longer second call duration than the non-toll-free
- There seems to be an outlier event in mid July, there was a very high count of numbers that called back with in a very short timespan

In [ ]:
df = client.query(sql.format("VA Emergency Outage Message")).to_dataframe()
plot_callbacks(df)

## Callbacks - VA Emergency Outage Message

The above plot show characteristics of phone numbers that have appeared in more than one SessionId. The top left is the count of how many of these repeat numbers called for the first time on that date. The top right plot shows the number of days between the two sessionIds and the bottom two plots display the duration in minutes for both the first call and the second. 

**TollFreeNumbers** is a subgroup of phone numbers that start with '+1800'

### Takeaways

- There seems to be an outlier event in mid July, there was a very high count of numbers that called back with in a very short timespan

In [ ]:
df = client.query(sql.format("PA Emergency Outage Message")).to_dataframe()
plot_callbacks(df)

## Callbacks - PA Emergency Outage Message

The above plot show characteristics of phone numbers that have appeared in more than one SessionId. The top left is the count of how many of these repeat numbers called for the first time on that date. The top right plot shows the number of days between the two sessionIds and the bottom two plots display the duration in minutes for both the first call and the second. 

**TollFreeNumbers** is a subgroup of phone numbers that start with '+1800'

### Takeaways

- There seems to be an outlier event in mid July, there was a very high count of numbers that called back with in a very short timespan

In [ ]:
df = client.query(sql.format("IL Emergency Outage Message")).to_dataframe()
plot_callbacks(df)

## Callbacks - PA Emergency Outage Message

The above plot show characteristics of phone numbers that have appeared in more than one SessionId. The top left is the count of how many of these repeat numbers called for the first time on that date. The top right plot shows the number of days between the two sessionIds and the bottom two plots display the duration in minutes for both the first call and the second. 

**TollFreeNumbers** is a subgroup of phone numbers that start with '+1800'

### Takeaways

- There seems to be an outlier event in mid July, there was a very high count of numbers that called back with in a very short timespan